In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

from copy import deepcopy
import os,time
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Loading data and preprocessing

import data

In [ ]:
train = pd.read_csv('/kaggle/input/recipe-for-rating-predict-food-ratings-using-ml/train.csv')
test = pd.read_csv('/kaggle/input/recipe-for-rating-predict-food-ratings-using-ml/test.csv')

train_text = train.Recipe_Review
test_text = test.Recipe_Review
train_text1 = train.RecipeName
test_text1 = test.RecipeName


train_labels = train['Rating']
train.drop(['Rating'],axis=1,inplace=True)

### Data Analysis

In [ ]:
for i in test.columns:
    print(i.ljust(20),(len(set(train[i])) , (set(train[i])) == (set(test[i]))))

### selecting required features

In [ ]:
to_drop = ['UserID','ID','UserName','CommentID','Recipe_Review','CreationTimestamp','RecipeCode']


numeric = ['RecipeNumber', 'UserReputation', 'ReplyCount','ThumbsUpCount', 'ThumbsDownCount', 'BestScore']
one_hot = []
ordinal = ['UserReputation','RecipeName']


train.drop(to_drop,axis=1,inplace = True)
test.drop(to_drop,axis=1,inplace = True)

In [ ]:
num_cols = [
    'RecipeNumber', 'UserReputation',
    'ReplyCount', 'ThumbsUpCount', 'ThumbsDownCount', 'BestScore'
]

from sklearn.preprocessing import MinMaxScaler

def hmp(x,y,tit):
    x_ = sorted(set(list(np.array(x))))
    y_ = sorted(set(y))
    df = pd.DataFrame(np.zeros((len(x_),len(y_))),columns = range(6),index = x_)
    for i,j in np.column_stack([x,y]):
        df.loc[i,j] += 1
    print(tit)
    sns.heatmap(df,cmap="YlGnBu")
    plt.show()

import matplotlib.pyplot as plt
import seaborn as sns

for i in num_cols:
    hmp(train[i],train_labels,i)

In [ ]:
# data preprocessing

from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,PolynomialFeatures
from sklearn.preprocessing import MaxAbsScaler,MinMaxScaler,StandardScaler

from sklearn.feature_extraction.text import TfidfVectorizer,HashingVectorizer,CountVectorizer

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import ShuffleSplit,KFold

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.impute import SimpleImputer
from sklearn.base import clone

from sklearn.base import BaseEstimator

In [ ]:
# metrics

from sklearn.metrics import ConfusionMatrixDisplay, zero_one_loss , classification_report

In [ ]:
preprocess = Pipeline(
    [
        (
            'ct',
            ColumnTransformer(
                [
                    ('ordinl' , OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=np.nan) , ordinal),
                    ('numeric' , StandardScaler() , numeric),
                ]
            )
        ),
        ('inp',SimpleImputer(strategy='mean')),
        ('scale' , StandardScaler(with_mean = True , with_std = True)),
        ('pf' , PolynomialFeatures(degree=3,interaction_only=True))
    ]
)
tf = TfidfVectorizer()

In [ ]:
train = pd.DataFrame(preprocess.fit_transform(train))
test = pd.DataFrame(preprocess.transform(test))

usefull = list(pd.DataFrame(train.copy()).sum(axis=0)!=0)
train = train.loc[:,(usefull)]
test = test.loc[:,(usefull)]

In [ ]:
train.columns = range(train.shape[1])
test.columns = range(train.shape[1])

X_train, X_test, y_train, y_test = train_test_split(train, train_labels, random_state= 42, test_size = 0.2,stratify=train_labels)

# Tests

Linear

In [ ]:
from sklearn.linear_model import SGDClassifier,LogisticRegression

In [ ]:
# sgd

sgdEst = SGDClassifier(random_state=75,max_iter=10000,eta0 = 0.005, learning_rate='adaptive')

sgd = GridSearchCV(sgdEst,cv=ShuffleSplit(random_state=0),
                   param_grid = {
                       'eta0':[0.01,0.1],
                       'penalty':['l1','l2'],
                       'power_t':[0.01,2]
                   },verbose=2,return_train_score=True,
                   scoring = 'f1_macro'
                  )

sgd.fit(X_train,y_train)

In [ ]:
print(classification_report(y_test,sgd.predict(X_test)))

In [ ]:
# logistic regression

log_reg = GridSearchCV(
    LogisticRegression(random_state=42,max_iter=10000,multi_class='ovr'),
    param_grid = {
        'penalty':['l2',None],'tol':[0.0001,0.01],
        'C':[1.0,100],
    'solver':['lbfgs'],
    },
    scoring = 'f1_macro',
    verbose = 3
)

log_reg.fit(X_train,y_train)
log_reg.score(X_test,y_test)
print(classification_report(y_test,log_reg.predict(X_test)))

In [ ]:
print(classification_report(y_train,log_reg.predict(X_train)))

KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = GridSearchCV(
    KNeighborsClassifier(),
    param_grid = {
        'n_neighbors' : [5 ,10 , 15 ],
        'weights' : ['uniform' , 'distance'],
        'leaf_size' : [6,7,8]
    },
    verbose = 3,
    scoring = 'f1_macro'
)
    

knn.fit(X_train,y_train)
print('train')
print(classification_report(y_train,knn.predict(X_train)))
print('test')
print(classification_report(y_test,knn.predict(X_test)))

SVM

In [ ]:
from sklearn.svm import SVC

svc = GridSearchCV(
    SVC(random_state=42,gamma='auto',max_iter=10000),
    param_grid = {
        'C':[1.0,100],'kernel':['rbf','poly'],
        'degree':[3,5],
    },
    verbose = 3,
    scoring = 'f1_macro'
)

svc.fit(X_train,y_train)
print('train\n',classification_report(y_train,svc.predict(X_train)))
print('test\n',classification_report(y_test,svc.predict(X_test)))

Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

gnb = GridSearchCV(
    GaussianNB(),
    param_grid = {
            'var_smoothing' : [1e-9,1e-8,1e-7],
            'priors':[{0:0.09326,1:0.01539,2:0.01275,3:0.02683,4:0.09106,5:0.76066},None]
    },
    verbose = 3
)

gnb.fit(X_train,y_train)
print('train\n',classification_report(y_train,gnb.predict(X_train)))
print('test\n',classification_report(y_test,gnb.predict(X_test)))

In [ ]:
print(classification_report(y_test,gnb.predict(X_test)))

# Model

#1 XGBoost

In [ ]:
from xgboost import XGBClassifier

clf1 = XGBClassifier(random_state=10 , max_depth = 100,)

clf1.fit(X_train,y_train)
clf1.score(X_test,y_test)

#2 KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

clf2 = KNeighborsClassifier(n_neighbors=25,leaf_size=1000,weights='distance',)

clf2.fit(X_train,y_train)
ConfusionMatrixDisplay.from_estimator(clf2,X_test,y_test)
clf2.score(X_test,y_test)

In [ ]:
print(classification_report(y_test,clf2.predict(X_test)))

#3 Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf3 = RandomForestClassifier(n_estimators=25,bootstrap=False,random_state=42,criterion='entropy')

clf3.fit(X_train,y_train)
ConfusionMatrixDisplay.from_estimator(clf3,X_test,y_test)
clf3.score(X_test,y_test)

In [ ]:
print(classification_report(y_test,clf3.predict(X_test)))

In [ ]:
ind = [i for i in range(1,4547)]
df = pd.DataFrame()
df['ID']=ind
df['Rating'] = clf1.predict(test)
df.to_csv('submission.csv',index=False)